In [8]:
import pandas as pd
import numpy as np

In [6]:
dat = pd.read_csv("hairdryer_cleaned.csv", header=0)

In [7]:
dat = dat[['product_parent','star_rating','review_date']]
dat

,product_parent,star_rating,review_date
0,732252283,5,8/31/2015
1,670161917,4,8/31/2015
2,16483457,5,8/31/2015
3,253917972,5,8/31/2015
4,732252283,4,8/31/2015
...,...,...,...
11106,919751065,5,8/21/2002
11107,694290590,1,8/13/2002
11108,694290590,5,7/13/2002
11109,694290590,5,4/20/2002


In [49]:
find = dat.groupby(['product_parent'], as_index=False)['star_rating'].count()
find.columns = ['review_date','nums']
find.sort_values("nums",inplace=True,ascending=False)
find

,review_date,nums
91,732252283,587
3,47684938,555
94,758099411,535
88,694290590,424
29,235105995,407
...,...,...
47,352847602,21
43,333479002,21
125,987021536,21
100,800605083,1


In [55]:
def getProductInfo(pid, j):
    subdat = dat[dat['product_parent'] == pid]
    subdat = subdat.reset_index(drop=True)
        
    tmp = subdat[subdat['star_rating'] == j]
    tmp = tmp.reset_index(drop=True)
    
    res = tmp.groupby(['review_date'], as_index=False)['star_rating'].count()
    res.columns = ['review_date','num_j']
    
    subdat = subdat.groupby(['review_date'], as_index=False)['star_rating'].count()
    subdat.columns = ['review_date','num']
    
    df = pd.merge(subdat, res, how='outer', on='review_date')
    df['review_date'] = pd.to_datetime(df['review_date'])
    df = df.sort_values(by = 'review_date')
    df.reset_index(inplace=True)
    del df['index']
    df = df.fillna(0)
    return df

In [56]:
see = getProductInfo(732252283, 5)
see.head(10)

,review_date,num,num_j
0,2010-11-22,1,1.0
1,2010-11-23,1,1.0
2,2010-11-25,1,0.0
3,2010-11-28,1,1.0
4,2010-12-13,1,1.0
5,2010-12-14,1,0.0
6,2010-12-15,2,2.0
7,2010-12-16,1,1.0
8,2010-12-21,1,0.0
9,2011-01-10,1,1.0


In [35]:
'''
see['cumsum'] = see['num'].cumsum()
see['cumsum_j'] = see['num_j'].cumsum()
see.head(10)
'''

,review_date,num,num_j,cumsum,cumsum_j
0,1/1/2013,1,0.0,1,0.0
1,1/1/2014,1,1.0,2,1.0
2,1/1/2015,1,1.0,3,2.0
3,1/10/2011,1,1.0,4,3.0
4,1/11/2015,2,1.0,6,4.0
5,1/12/2013,1,1.0,7,5.0
6,1/12/2015,2,2.0,9,7.0
7,1/13/2013,2,2.0,11,9.0
8,1/13/2015,2,1.0,13,10.0
9,1/14/2013,1,1.0,14,11.0


In [38]:
from statsmodels.tsa.stattools import grangercausalitytests

In [59]:
for j in range(1,6):
    
    # tmp = getProductInfo(732252283, j)
    
    tmp = getProductInfo(47684938, j)
    # tmp['cumsum'] = tmp['num'].cumsum()
    # tmp['cumsum_j'] = tmp['num_j'].cumsum()
    
    res = tmp[['num','num_j']]
    res.to_csv("47684938_%d.csv" % j, index=0)